In [18]:
import imageio
import glob
import numpy as np

osize = 640
tsize = 128
l = int((osize - tsize)/2)
h = l + tsize
x_train_o = np.zeros((112, tsize, tsize))
ind = 0

for im_path in glob.glob("PNG_Brodatz/*.png"):
    im = np.array(imageio.imread(im_path))
#     print(np.shape(im))
    im = im[l:h, l:h]
#     print(np.shape(im))
    x_train_o[ind] = im
#     print(im)
    ind = ind+1
#     print("hi")
     
    

print(np.shape(x_train_o))
from PIL import Image
img = Image.fromarray(x_train_o[0])
# img.show()
# print(x_train_o[0])

#  !pip install mat4py
from mat4py import loadmat

data = loadmat('labels.mat')

y_train = data['is_horizontal']
np.shape(y_train)
print(np.sum(y_train))

import numpy as np
print(np.shape(x_train_o))
print(np.shape(y_train))
x_train = np.reshape(x_train_o, (112, tsize*tsize))
y_train = np.reshape(y_train, 112)
print(np.shape(x_train))

from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)

ns = 70
x_traing = x_train[0:ns][:]
y_traing = y_train[0:ns][:]

y_traing

for x in np.unique(y_traing):
  print(np.shape(y_traing[y_traing==x])[0]/ns)

(112, 128, 128)
25
(112, 128, 128)
(112, 1)
(112, 16384)
0.7571428571428571
0.24285714285714285


In [2]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

kfold = StratifiedKFold(n_splits=10)

svclassifier = SVC()
svc_param_grid = {'kernel': ['linear'],
                  'C': [0.01, 0.1, 1, 10, 50, 100, 200, 300, 1000]}
gsSVM = GridSearchCV(svclassifier,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsSVM.fit(x_traing, y_traing)

SVM_best = gsSVM.best_estimator_
print(gsSVM.best_score_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s


0.7857142857142857


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    2.1s finished


In [3]:
train_data=np.sum(SVM_best.predict(x_traing) == y_traing)/ns
print(train_data)

1.0


In [4]:
# nt = 1000
nt = 112-ns
x_test = x_train[ns:][:]
y_test = y_train[ns:][:]
print(x_test.shape)
print(y_test.shape)

(42, 16384)
(42,)


In [5]:
test_data=np.sum(SVM_best.predict(x_test) == y_test)/nt
print(test_data)

0.7619047619047619


In [6]:
SVM_best.predict(x_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
y_test

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1])

In [8]:
n = tsize*tsize
m = 2**11
A = np.random.randn(m,n)
B=np.random.binomial(1,0.5,(m,n))
B[np.where(B==0)]=-1
# x_train_c = np.matmul(x_train, A.T)/m**0.5
# x_test_c = np.matmul(x_test, A.T)/m**0.5
x_train_g = np.matmul(x_traing, A.T)/m**0.5
x_test_g = np.matmul(x_test, A.T)/m**0.5
x_train_b = np.matmul(x_traing, B.T)/m**0.5
x_test_b = np.matmul(x_test, B.T)/m**0.5
print(np.shape(x_train_b))
print(np.shape(x_train_g))

(70, 2048)
(70, 2048)


In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

kfold = StratifiedKFold(n_splits=10)

svclassifier = SVC()
svc_param_grid = {'kernel': ['linear'],
                  'C': [0.01, 0.1, 1, 10, 50, 100, 200, 300, 1000]}
gsSVM_g = GridSearchCV(svclassifier,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsSVM_g.fit(x_train_g, y_traing)

SVM_best_g = gsSVM_g.best_estimator_
print(gsSVM_g.best_score_)

gsSVM_b = GridSearchCV(svclassifier,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsSVM_b.fit(x_train_b, y_traing)

SVM_best_b = gsSVM_b.best_estimator_
print(gsSVM_b.best_score_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    0.2s finished


0.7714285714285714
Fitting 10 folds for each of 9 candidates, totalling 90 fits
0.7714285714285714


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    0.2s finished


In [10]:
train_measurement_gaussian=np.sum(SVM_best_g.predict(x_train_g) == y_traing)/ns
print(train_measurement_gaussian)
train_measurement_bernoulli=np.sum(SVM_best_b.predict(x_train_b) == y_traing)/ns
print(train_measurement_bernoulli)

1.0
1.0


In [11]:
test_measurement_gaussian=np.sum(SVM_best_g.predict(x_test_g) == y_test)/nt
print(test_measurement_gaussian)
test_measurement_bernoulli=np.sum(SVM_best_b.predict(x_test_b) == y_test)/nt
print(test_measurement_bernoulli)

0.7142857142857143
0.7142857142857143


In [12]:
print("Stats for Brodatz dataset")
print("No. of Training Samples = ",ns)
print("No. of Testing Samples = ",nt)
print("Dimensions of image ",tsize,"*",tsize)
print("Number of features in data domain (originally) = ",tsize*tsize)
print("Number of measurements taken (No. of features in measurement domain) = ",m)
print("Train accuracy in original domain = ", train_data)
print("Test accuracy in original domain = ", test_data)
print("Train accuracy when random Gaussian matrix used ", train_measurement_gaussian)
print("Test accuracy when random Gaussian matrix used ", test_measurement_gaussian)
print("Train accuracy when random Bernoulli matrix used", train_measurement_bernoulli)
print("Test accuracy when random Bernoulli matrix used", test_measurement_bernoulli)

Stats for Brodatz dataset
No. of Training Samples =  70
No. of Testing Samples =  42
Dimensions of image  128 * 128
Number of features in data domain (originally) =  16384
Number of measurements taken (No. of features in measurement domain) =  2048
Train accuracy in original domain =  1.0
Test accuracy in original domain =  0.7619047619047619
Train accuracy when random Gaussian matrix used  1.0
Test accuracy when random Gaussian matrix used  0.7142857142857143
Train accuracy when random Bernoulli matrix used 1.0
Test accuracy when random Bernoulli matrix used 0.7142857142857143


In [13]:
SVM_best_g.predict(x_test_g)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [14]:
SVM_best_b.predict(x_test_b)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [15]:
y_test

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1])

In [16]:
print(x_traing.shape)
print(x_test.shape)
print(x_train_b.shape)
print(x_test_b.shape)


(70, 16384)
(42, 16384)
(70, 2048)
(42, 2048)
